**Notebook for the paper**<br/>
**_A churn prediction dataset from the telecom sector: a new benchmark for uplift modeling_**<br/>
_Anonymous authors_
# Benchmark for uplift models on the Criteo dataset

In [ ]:
%load_ext autoreload
%autoreload 2
import pickle
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import normalize
from os.path import join
from scipy.stats import entropy

from functions.wrappers import URFCWrapper, RandomForestWrapper, SLearnerWrapper
from functions.benchmark import benchmark
from functions.dataset import Dataset
from functions.easy_ensemble import EasyEnsemble
from functions.eval_measures import uplift_curve, calibrate_score

### Data preparation

In [ ]:
data = pd.read_csv("../data/criteo-uplift-v2.1.csv")

In [ ]:
data = data.iloc[np.random.choice(data.shape[0], replace=False, size=50000)]

In [ ]:
exclude_from_X = ['treatment', 'conversion', 'visit', 'exposure']
columns_X = [c for c in data.columns if c not in exclude_from_X]
dataset = Dataset(
    X = data[columns_X].reset_index(drop=True).to_numpy(),
    y = (data.visit == 1).to_numpy(),
    t = (data.treatment == 1).to_numpy()
)

In [ ]:
# Data normalization
normalization = "minmax"
if normalization == "gaussian":
    m = dataset.X.mean(axis=0)
    s = dataset.X.std(axis=0)
    dataset.X = (dataset.X - m) / s
elif normalization == "minmax":
    M = dataset.X.max(axis=0)
    m = dataset.X.min(axis=0)
    dataset.X = (dataset.X - m) / (M - m)

# Target variables
dataset.t = np.array(dataset.t)
dataset.y = np.array(dataset.y)

In [ ]:
U = dataset.y[~dataset.t].mean() - dataset.y[dataset.t].mean()

In [ ]:
print("Uplift in this dataset: {:.2%}".format(U))

In [ ]:
with open("../data/prepared_criteo.pickle", 'wb') as f:
    pickle.dump(dataset, f)

### Benchmark setup

In [ ]:
with open("../data/prepared_criteo.pickle", "rb") as f:
    dataset = pickle.load(f)

In [ ]:
seed = None

In [ ]:
models = {}
max_features = int(sqrt(dataset.X.shape[1]))
n_estimators = 100
max_depth = 20
min_samples_leaf = 10
n_folds = 8

# RFeature with uplift random forest and EasyEnsemble
models["urf_rfeature"] = RFeature(
    RandomForestClassifier(n_estimators=n_estimators, n_jobs=-1, random_state=seed),
    EasyEnsemble(
        URFCWrapper(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            min_samples_treatment=min_samples_leaf,
            max_features=max_features,
            n_jobs=1,
            random_state=seed
        ),
        n_folds=n_folds,
        n_jobs=-1,
        random_state=seed,
        verbose=False
    ),
    verbose=False
)

# Uplift random forest and EasyEnsemble without rfeature
models["urf"] = EasyEnsemble(
    URFCWrapper(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        min_samples_treatment=min_samples_leaf,
        max_features=max_features,
        n_jobs=1,
        random_state=seed
    ),
    n_folds=n_folds,
    n_jobs=-1,
    random_state=seed,
    verbose=False
)

# RFeature with X-learner and EasyEnsemble
models["xlearner_rfeature"] = RFeature(
    RandomForestClassifier(n_estimators=100, n_jobs=-1),
    EasyEnsemble(
        XClassifierWrapper(
            outcome_learner=RandomForestClassifier(n_estimators=n_estimators, n_jobs=1),
            effect_learner=RandomForestRegressor(n_estimators=n_estimators, n_jobs=1)
        ),
        n_folds=n_folds,
        n_jobs=-1,
        random_state=seed,
        verbose=False
    ),
    verbose=True
)

# RFeature with X-learner and EasyEnsemble
models["xlearner"] = EasyEnsemble(
    XClassifierWrapper(
        outcome_learner=RandomForestClassifier(n_estimators=100, n_jobs=1),
        effect_learner=RandomForestRegressor(n_estimators=100, n_jobs=1)
    ),
    n_folds=n_folds,
    n_jobs=-1,
    random_state=seed,
    verbose=False
)

# Causal effect variational autoencoder
models["cevae_rfeature"] = RFeature(
    RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=seed),
    CEVAEWrapper(),
    verbose=False
)

# Churn risk
models["rf_rfeature"] = RFeature(
    RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=seed),
    EasyEnsemble(
        RandomForestWrapper(
            n_jobs=1,
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            random_state=seed
        ),
        n_folds=n_folds,
        n_jobs=-1,
        random_state=seed,
        verbose=False
    ),
    verbose=False
)

# Churn risk no reach
models["rf"] = EasyEnsemble(
    RandomForestWrapper(
        n_jobs=1,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=seed
    ),
    n_folds=n_folds,
    n_jobs=-1,
    random_state=seed,
    verbose=False
)

# RF S learner
models["rf_slearner"] = EasyEnsemble(
    SLearnerWrapper(
        n_jobs=1,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=seed
    ),
    n_folds=n_folds,
    n_jobs=-1,
    random_state=seed,
    verbose=False
)

models_to_run = ["rf", "rf_slearner", "urf"]
models = {name: models[name] for name in models_to_run}

In [ ]:
results_file = "results/benchmark_criteo.pickle"

In [ ]:
n_repeats = 10
k_folds = 3

### Benchmark

In [ ]:
results = benchmark(
    dataset,
    models=models,
    k_folds=k_folds,
    n_repeats=n_repeats,
    seed=None,
    verbose=False,
    predict_params={
        "urf_rfeature": {"full_output": True},
        "urf": {"full_output": True}
    }
)
if True:
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)

In [ ]:
with open(results_file, "rb") as f:
    results = pickle.load(f)

### Estimating the mutual information and the estimator variance

First, build the array of predictions of each model, to compute the variance

In [ ]:
preds = {}
for model_name in models:
    all_pred = np.empty((dataset.X.shape[0], n_repeats))
    for i, split in enumerate(results):
        i_repeat = i // k_folds
        pred = split["results"][model_name]["pred"]
        if model_name == "urf" or model_name == "rf_slearner":
            pred = pred["control"] - pred["target"]
        all_pred[split["test_indices"], i_repeat] = pred
    all_pred[np.isnan(all_pred)] = 0
    all_pred[all_pred > 1e10] = 0 # Sometimes some models give huge scores
    preds[model_name] = all_pred

In [ ]:
def estimator_variance(preds):
    preds[preds <= -1] = -1
    preds[preds >= 1] = 1
    preds[np.isnan(preds)] = 0
    return np.mean(np.var(preds, axis=1))

In [ ]:
variances = {model_name: estimator_variance(preds[model_name]) for model_name in models}

In [ ]:
#print("Var_u = {:.2%}".format(variances["urf"]))
#print("Var_p = {:.2%}".format(variances["rf"]))
print("Var_s = {:.2e}".format(variances["rf_slearner"]))

Then, estimate the individual counterfactual probabilities to estimate the mutual information

In [ ]:
S_0_prior = np.mean(dataset.y[~dataset.t])
S_1_prior = np.mean(dataset.y[dataset.t])

In [ ]:
S_0 = np.empty((dataset.X.shape[0], n_repeats))
S_1 = np.empty((dataset.X.shape[0], n_repeats))

for i, split in enumerate(results):
    i_repeat = i // k_folds
    S_0_split = split["results"]["rf_slearner"]["pred"]["control"]
    S_1_split = split["results"]["rf_slearner"]["pred"]["target"]
    #S_0_split = calibrate_score(S_0_split, S_0_prior)
    #S_1_split = calibrate_score(S_1_split, S_1_prior)
    S_0[split["test_indices"], i_repeat] = S_0_split
    S_1[split["test_indices"], i_repeat] = S_1_split

S_0[np.isnan(S_0)] = 0
S_1[np.isnan(S_1)] = 0
S_0[S_0 >= 1] = 1
S_1[S_1 >= 1] = 1
S_0[S_0 <= 0] = 0
S_1[S_1 <= 0] = 0
S_0 = np.mean(S_0, axis=1)
S_1 = np.mean(S_1, axis=1)

In [ ]:
# Calibrate two times for more precision
S_0 = calibrate_score(calibrate_score(S_0, S_0_prior), S_0_prior)
S_1 = calibrate_score(calibrate_score(S_1, S_1_prior), S_1_prior)

In [ ]:
def counterfactuals(S_0_x, S_1_x):
    alpha_x = (1 - S_0_x) * (1 - S_1_x)
    beta_x = S_0_x * (1 - S_1_x)
    gamma_x = (1 - S_0_x) * S_1_x
    delta_x = S_0_x * S_1_x
    alpha = np.mean(alpha_x)
    beta = np.mean(beta_x)
    gamma = np.mean(gamma_x)
    delta = np.mean(delta_x)
    return (alpha, beta, gamma, delta)

def mutual_information_marginal(S_x, S):
    return entropy([S, 1 - S]), np.mean(entropy(np.vstack((S_x, 1 - S_x))))

In [ ]:
alpha, beta, gamma, delta = counterfactuals(S_0, S_1)
print("alpha = {:.1%}".format(alpha))
print("beta  = {:.1%}".format(beta))
print("gamma = {:.1%}".format(gamma))
print("delta = {:.1%}".format(delta))

In [ ]:
H_Y_0, H_Y_0_X = mutual_information_marginal(S_0, S_0_prior)
H_Y_1, H_Y_1_X = mutual_information_marginal(S_1, S_1_prior)

In [ ]:
print("I_0 = {:.2%}".format(1 - H_Y_0_X / H_Y_0))
print("I_1 = {:.2%}".format(1 - H_Y_1_X / H_Y_1))

### Plotting uplift curves

In [ ]:
curves = {}
auucs = {}

auucs["random"] = (S_0_prior - S_1_prior) / 2
N = dataset.X.shape[0]

for model_name in models:
    curves[model_name] = []
    auucs[model_name] = []
    for i, split in enumerate(results):
        i_repeat = i // k_folds
        pred = split["results"][model_name]["pred"]
        if model_name.startswith("urf") or model_name == "rf_slearner":
            pred = pred["control"] - pred["target"]
        if model_name in ("urf_rfeature", "urf", "cevae", "xlearner", "xlearner_rfeature", "rf_slearner"):
            pred = -pred
        pred[np.isnan(pred)] = 0
        pred[pred > 1e10] = 0 # Sometimes some models give huge scores
        curve = uplift_curve(dataset.y[split["test_indices"]], dataset.t[split["test_indices"]], pred)
        curves[model_name].append(curve)
        auucs[model_name].append(-curve.profit.mean() / len(split["test_indices"]))

In [ ]:
#print("AUUC_u = {:.2%}".format(np.mean(auucs["urf"])))
#print("AUUC_p = {:.2%}".format(np.mean(auucs["rf"])))
print("AUUC_p = {:.2%}".format(np.mean(auucs["rf_slearner"])))